In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import theano
import theano.tensor as T

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
SEQ_LENGTH = 1000
def generate_single_example(power, duration, start):
    activation = np.ones(duration, dtype=np.float32) * power
    sequence = np.zeros(SEQ_LENGTH, dtype=np.float32)
    sequence[start:start+duration] += activation
    return sequence

In [13]:
N_SEQ_PER_BATCH = 5
def generate_example(*args, **kwargs):
    data = np.empty((N_SEQ_PER_BATCH, SEQ_LENGTH, 1))
    for i in range(N_SEQ_PER_BATCH):
        data[i,:,0] = generate_single_example(*args, **kwargs)
    return data

In [4]:
error = 50
power = 100

short_seq_t = generate_single_example(power=power, duration=100, start=100)
short_seq_x = generate_single_example(power=power+error, duration=100, start=100)

long_seq_t = generate_single_example(power=power, duration=500, start=100)
long_seq_x = generate_single_example(power=power+error, duration=500, start=100)

In [5]:
def cost(x, t):
    return np.mean((x - t) ** 2)

In [6]:
cost(short_seq_t, short_seq_x)

250.0

In [6]:
cost(long_seq_t, long_seq_x)

1250.0

In [7]:
short_seq_t[short_seq_t > 0]

array([ 100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.,  100.], dtype=float32)

In [48]:
from theano.ifelse import ifelse
THRESHOLD = 0
def cost2(x, t):
    sq_error = (x - t) ** 2
    def mask_and_mean_sq_error(mask):
        masked_sq_error = sq_error[mask.nonzero()]
        mean = masked_sq_error.mean()
        mean = ifelse(T.isnan(mean), 0.0, mean)
        return mean
    above_thresh_mean = mask_and_mean_sq_error(t > THRESHOLD)
    below_thresh_mean = mask_and_mean_sq_error(t <= THRESHOLD)
    return (above_thresh_mean + below_thresh_mean) / 2.0

In [49]:
cost2(theano.shared(short_seq_x), theano.shared(short_seq_t)).eval()

array(1250.0, dtype=float32)

In [50]:
cost2(short_seq_x, short_seq_t)

Elemwise{true_div,no_inplace}.0

In [51]:
cost2(theano.shared(np.zeros(TOTAL_LENGTH, dtype=np.float32)), theano.shared(np.zeros(TOTAL_LENGTH, dtype=np.float32))).eval()

array(0.0, dtype=float32)

In [27]:
(t > THRESHOLD).shape[0].eval()

array(1000)

In [35]:
(0-0) ** 2

0

In [14]:
t = generate_example(power=power, duration=100, start=100)

In [19]:
n_above_thresh_per_seq = (t > 0).sum(axis=1)

In [20]:
n_below_thresh_per_seq = SEQ_LENGTH - n_above_thresh_per_seq

In [22]:
mask = np.ones(t.shape)

In [24]:
mask[(t > 0)].shape

(500,)

In [27]:
(t > 0).shape

(5, 1000, 1)